# TensorBoardを用いた埋め込みの可視化

本ノートブックでは、TensorBoardを用いて埋め込みの可視化をします。
埋め込みの可視化は、以下のサイトから確認することもできます。

- [Embedding projector](https://projector.tensorflow.org/)

## パッケージのインポート

In [1]:
import os
import warnings

import numpy as np
import tensorflow as tf
from gensim.models import KeyedVectors
from tensorboard.plugins import projector
warnings.filterwarnings('ignore')

## 埋め込みのアップロード

In [2]:
from google.colab import files
uploaded = files.upload()

Saving word2vec_cbow.bin to word2vec_cbow.bin


## 埋め込みの読み込み

In [3]:
filename = "word2vec_cbow.bin"
model = KeyedVectors.load_word2vec_format(filename, binary=True)

## TensorBoard用にデータを保存

TensorBoardは、TensorFlowプロジェクトのログからテンソルとメタデータを読み込みます。今回は、TensorBoardにデータを読み込むために、可視化用のメタデータと埋め込みを保存します。

In [14]:
# ボキャブラリ数の取得
max_size = len(model.wv.vocab) - 1

In [15]:
# ボキャブラリ数x次元数のnumpy配列を作成
w2v = np.zeros((max_size, model.wv.vector_size))

In [16]:
# metadata.tsvの作成し、すべての単語を書き込む
# また、各単語の埋め込みをw2v行列に格納
log_dir = 'logs/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

with open(os.path.join(log_dir, "metadata.tsv"), 'w') as file_metadata:
    for i, word in enumerate(model.wv.index2word[:max_size]):
        # 単語埋め込みを格納
        w2v[i] = model.wv[word]
        # 単語をファイルに書き込み
        file_metadata.write(f'{word}\n')

In [17]:
# 分析する埋め込みを変数として保存
weights = tf.Variable(w2v, trainable=False, name='embedding')

# 埋め込みからチェックポイントを作成
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

# プロジェクタの初期化と埋め込みの追加
config = projector.ProjectorConfig()
embedding = config.embeddings.add()

# テンソル名とメタデータのパスを指定
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

## TensorBoardでの可視化

トップメニューの「INACTIVE」をクリックして、「PROJECTIONS」を選択します。

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./logs

## gensimを用いたデータ変換

TensorBoardには、以下のようなTSV形式でベクトルとメタデータをアップロードすると、埋め込みを可視化できる機能があります。

ベクトルの形式

```bash
0.1\t0.2\t0.5\t0.9
0.2\t0.1\t5.0\t0.2
0.4\t0.1\t7.0\t0.8
```

メタデータの形式。複数の列がある場合、最初の行が列のラベルとして解釈される。

```bash
Pokémon\tSpecies
Wartortle\tTurtle
Venusaur\tSeed
Charmeleon\tFlame
```

gensimで学習したword2vecモデルは、[scripts.word2vec2tensor](https://radimrehurek.com/gensim/scripts/word2vec2tensor.html)を使うことで、TensorBoard形式へ簡単に変換できます。手順は以下の通りです。

1. スクリプトを実行して、モデルを変換する
```bash
python -m gensim.scripts.word2vec2tensor -i word2vec_cbow.bin -o embedding -b
```
2. [http://projector.tensorflow.org/](http://projector.tensorflow.org/)を開く
3. Projectorの画面左メニューにある「Load Data」をクリック
4. 「Load a TSV file of vectors.」のセクションにある「Choose file」を選択して、ベクトルのファイルをアップロード
5. 「Load a TSV file of metadata.」のセクションにある「Choose file」を選択して、メタデータをアップロード